# Import modules

In [1]:
import pandas as pd
import geopandas as gpd
from bng_latlon import OSGB36toWGS84

# Read data

In [2]:
# Read the stops data
stops = pd.read_csv(
        "../../../pipelines/transport/stops/data/stops.csv",
        usecols=['ATCOCode', 'NaptanCode', 'Latitude', 'Longitude', 'Easting', 'Northing', 'StopType', 'BusStopType', 'Status'], 
        low_memory=False
        )

# Read the constituency geoJSON
constituency_gdf = gpd.read_file("../../../pipelines/transport/stops/data/Westminster_Parliamentary_Constituencies_July_2024_Boundaries_UK_BGC_-8097874740651686118.geojson")
# Filter the two columns we need
constituency_gdf = constituency_gdf[['PCON24CD', 'geometry']].copy()

# Fill in missing Lat/long using Easting and Northing

In [3]:
# Apply function row-wise
stops[['Latitude', 'Longitude']] = stops.apply(
    lambda row: OSGB36toWGS84(row['Easting'], row['Northing']) if pd.isna(row['Latitude']) or pd.isna(row['Longitude']) else (row['Latitude'], row['Longitude']), 
    axis=1, result_type="expand"
)

# Data cleansing

In [4]:
# Drop duplicates in NaptanCode and coordinates
stops.drop_duplicates(subset=['Latitude', 'Longitude', 'NaptanCode'], inplace=True)
# Get active stops only
stops = stops[stops.Status == 'active'].copy()

## Prepare, analyse and reshape data

In [5]:
def stops_to_constituencies(stops, constituency_gdf, out):
    ## Convert Lat/Long to a GeoDataFrame using points_from_xy
    geometry = gpd.points_from_xy(stops['Longitude'], stops['Latitude'])
    ## Create the geodataframe
    points_gdf = gpd.GeoDataFrame(stops, geometry=geometry, crs=constituency_gdf.crs)
    ## Spatial join to find which geometry each point falls into
    joined = gpd.sjoin(points_gdf, constituency_gdf, how='left', predicate='within')

    # Reproject to an appropriate CRS
    constituency_gdf = constituency_gdf.to_crs(epsg=6933)
    
    # Calculate area in square kilometres
    constituency_gdf["area_km2"] = round(constituency_gdf.geometry.area / 1e6, 2)

    ## Group the data and value count
    group = joined.groupby(['StopType'])['PCON24CD'].value_counts().reset_index()
    
    ## Pivot data
    wide = group.pivot(index='PCON24CD', columns='StopType', values='count')

    ## Add area
    wide = wide.merge(constituency_gdf[['PCON24CD', 'area_km2']], how='inner', on='PCON24CD')
    wide.rename(columns={'BCT': 'bus_stops', 'MET': 'tram_metro_stops', 'RLY': 'rail_stops'}, inplace=True)
    ## Calculate density
    wide['bus_stops_density'] = round(wide['bus_stops'] / wide['area_km2'], 2)
    wide['tram_metro_stops_density'] = round(wide['tram_metro_stops'] / wide['area_km2'], 2)
    wide['rail_stops_density'] = round(wide['rail_stops'] / wide['area_km2'], 3)

    ## Calculate total number of all stop types
    wide['alltypes'] = joined['PCON24CD'].value_counts()

    ## Set the index
    wide.set_index('PCON24CD', inplace=True)

    ## Write to file
    wide.to_csv(out)

## Write to file

In [6]:
stops_to_constituencies(stops, constituency_gdf, "../../../src/themes/transport/transport-stops/_data/release/all.csv")

## Stop types

### BusCoach

- BCT: On-street Bus/Coach/Trolley
- BCE: Bus/Coach station entrance
- BST: Bus/coach access area
- BCS: Bus/Coach Bay/stand etc
- BCQ: Bus Coach statopm variable bay

### Taxi

- TXR: Taxi rank
- STR: shared Taxi Rank

### Car

- SDA: Set down area

### Air

- AIR: airport entrance
- GAT: airport interchange area

### Rail

- RSE: railway entrance
- RLY: railway interchange area
- RPL: Railway platform

### Tram/Metro

- TMU: tram/metro/underground entrance
- MET: unerground or metro interchange area
- PLT: underground or metro platform